In [1]:
import pandas as pd

BBSI = '/kaggle/input/krom-bank-indonesia-stock-historical-price/BBSI.JK.csv'

# there are no adjustments so we can drop the adjusted close column
df = pd.read_csv(filepath_or_buffer=BBSI, parse_dates=['Date']).drop(columns=['Adj Close'])
df['year'] = df['Date'].dt.year
df.head()

,Date,Open,High,Low,Close,Volume,year
0,2020-09-07,549.837036,549.837036,458.197540,549.837036,29516635,2020
1,2020-09-08,595.656799,687.296326,595.656799,687.296326,19232861,2020
2,2020-09-09,733.116089,856.829407,687.296326,765.189880,33271020,2020
3,2020-09-10,765.189880,797.263733,714.788147,714.788147,1050747,2020
4,2020-09-11,668.968384,668.968384,668.968384,668.968384,500439,2020


In [2]:
from plotly.express import scatter
scatter(data_frame=df, x='Date', y='Volume', log_y=True, trendline='lowess')

In [3]:
df['Volume'].max(), df[df['Volume'] > 0]['Volume'].min()

(33271020, 100)

We have a wide range of volume amounts; it's hard to believe how much it varies.

In [4]:
from plotly.express import line
line(data_frame=df, x='Date', y=df.columns[1:-2])

Our price data clearly has two periods where prices behaved differently: a run-up through August 2021 and a gradual decline thereafter. The trend becomes clearly if we look at moving averages.

In [5]:
from plotly.express import scatter
for rule in ['W', 'ME', 'QE']:
    line(data_frame=df.set_index(keys=['Date']).resample(rule=rule).mean().reset_index(), x='Date', y=df.columns[2:-2]).show()

Let's see how much money we could make trading BBSI.

In [6]:
# what are the starting and ending values and the difference between them?
df['Close'].values[0], df['Close'].values[-1], df['Close'].values[-1] - df['Close'].values[0] 

(549.8370361328125, 3670.0, 3120.1629638671875)

In [7]:
total_return = df['Close'].values[-1]/df['Close'].values[0]
print('total return: {:5.2f} pct'.format(100 * total_return))
years = (df['Date'].max() - df['Date'].min()).days / 365 # no leap years
print('years: {:5.2f}'.format(years))

total return: 667.47 pct
years:  3.42


In [8]:
from math import log
from math import exp

def get_rate(start_value: float, end_value: float, years: float) -> float:
    rate = exp(1.0/years * log(end_value/start_value)) - 1.0
    return rate

annualized_rate = get_rate(start_value=df['Close'].values[0], end_value=df['Close'].values[-1], years=years)

print('annualized pct rate: {:6.3f}'.format(100 * annualized_rate))

annualized pct rate: 74.229


An annualized return of 74% looks great; it's going to be hard to beat buy and hold with an overall annualized return that high.

In [9]:
from numpy import nan
from numpy import where

def strategy(input_df: pd.DataFrame, quote: str, long: int, short: int) -> float:
    work_df = input_df.copy()
    work_df['short'] = work_df[quote].rolling(window=short).mean()
    work_df['long'] = work_df[quote].rolling(window=long).mean()
    work_df['signal'] = where(work_df['short'] > work_df['long'], 1, 0)
    work_df['position'] = work_df['signal'].diff().fillna(value=1) # buy on day one
    work_df['buy'] = where(work_df['position'] == 1, work_df[quote], nan)
    work_df['sell'] = where(work_df['position'] == -1, work_df[quote], nan)
    return work_df['sell'].sum() - work_df['buy'].sum(), (work_df['position'] != 0).sum()

strategy(input_df=df, quote='Close', long=200, short=50)

(-2122.6063232421875, 4)

We have a real problem here because we need 200 days to get our 200d moving average off the ground, and by then the run-up phase is (almost?) over. Let's see if we can find any strategy that makes a profit.

In [10]:
short = []
long = []
profit = []
trades = []
for short_value in range(4, 124, 4):
    for long_value in range(short_value + 4, 252, 4):
        short.append(short_value)
        long.append(long_value)
        value, count = strategy(input_df = df[['Close']], quote='Close', long=long_value, short=short_value,)
        profit.append(value)
        trades.append(count)
size = [abs(item) for item in profit]
plot_df = pd.DataFrame(data={'short': short, 'long': long, 'profit/loss': profit, 'size': size, 'trades': trades})
scatter(x='short', y='long', color='profit/loss', size='size', height=900, data_frame=plot_df, hover_data=['trades']).show()
scatter(x='short', y='long', color='trades', size='trades', height=900, data_frame=plot_df, hover_data=['profit/loss']).show()

Not many strategies make a profit; and most of those look like buy and hold most of the time. What percentage of our strategies make a profit?

In [11]:
print('{:5.2f} pct of our strategies make a profit.'.format(100 * len(plot_df[plot_df['profit/loss'] > 0]) / len(plot_df)))

 7.10 pct of our strategies make a profit.


Before we go let's look at how price and volume correlate; we expect there to be essentially no correlation.

In [12]:
scatter(data_frame=df, x='Volume', y='Close', color='year', log_x=True)

Crazy how this stock has prices that have behaved sensibly on such low volume on some days in 2022-2024.